In [45]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize, linemerge
import Outils
#pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 50)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# IMPORTER LES DONNEES 
> depuis la base de données locale, en ne prenant en compte que les lignes qui intersectent une zone carré au N-O

In [3]:
#importer les odnnées de  base
with ct.ConnexionBdd('local_otv') as c : 
    requete="""with jointure as (
        select t.*, v1.cnt nb_intrsct_src, st_astext(v1.the_geom) as src_geom, v2.cnt as nb_intrsct_tgt, st_astext(v2.the_geom) as tgt_geom 
         from public.traf2015_bdt17_ed15_l t 
        left join public.traf2015_bdt17_ed15_l_vertices_pgr v1 on t.source=v1.id 
        left join public.traf2015_bdt17_ed15_l_vertices_pgr v2  on t.target=v2.id
        )
        select j.* from jointure j, zone_test_agreg z
        where st_intersects(z.geom, j.geom)"""

    df = gp.read_postgis(requete, c.connexionPsy)

>Mise en forme et définition des voies à 2 chaussées

In [4]:
nature_2_chaussees=['Route à 2 chaussées','Quasi-autoroute','Autoroute']
#donnees generale du jdd
#df.set_index('id_ign', inplace=True)  # passer l'id_ign commme index ; necessaire sinon pb avec geometrie vu comme texte
df2_chaussees=df.loc[df['nature'].isin(nature_2_chaussees)] #isoler les troncon de voies decrits par 2 lignes

# TRAITEMENT DES RONDS POINTS

In [5]:
#recuperer les lignes qui constituent les rd points
lgn_rd_pt=at.lignes_rd_pts(df)

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [6]:
#trouver les lignes entrantes sur les rd points : celle qui intersectent le poly ext mais pas le poly int d'un rd pt et qui ne sont pas ds ligne rd pt
ligne_entrant_rd_pt=at.lign_entrant_rd_pt(df,lgn_rd_pt)

In [7]:
#caractériser les rd points
carac_rd_pt=at.carac_rond_point(ligne_entrant_rd_pt)
#sauver le fichier ?

In [42]:
#ajouter l'identifiant du rd point aux données BdTopo
df_avec_rd_pt=df.merge(lgn_rd_pt[['id_ign', 'id_rdpt']], how='left', on='id_ign').merge(carac_rd_pt.reset_index(), how='left', on='id_rdpt')

In [46]:
df_avec_rd_pt.loc[df_avec_rd_pt['id_rdpt']>0]

,id_ign,geom,id,nature,numero,importance,cl_admin,gestion,mise_serv,fictif,franchisst,largeur,z_ini,z_fin,nb_voies,sens,etat,long_km,coment_cpt,src_cpt,id_comptag,obs_supl,ann_pt,coment_tmj,tmja,...,obs_vts,vts_osm,vts_modif,vts_vl_f,vts_pl_f,codau,codau_cont,tmja_cont,pc_pl_cont,source,target,recup,attr_modif,codevoie_d,id_tronc_elem,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt,codevoie_rdpt,nb_obj_sig_entrant,valeur_sens
54,TRONROUT0000000100634026,"(LINESTRING (399180.7999999997 6533043.299999991, 399179.0999999996 6533041.99999999, 399178.2999999996 6533039.09999999, 399178.9999999997 6533036.799999992))",210282,Route à 1 chaussée,D117,3,Départementale,17,NR,Non,NC,5.0,21.0,20.9,1.0,Direct,NR,0.007553,linearisation,otv,17-D117-27+400,None,2015,/2,1635.0,...,None,None,None,57.0,52.0,None,None,None,None,86065,86825,None,None,NR,NaN,3,POINT(399180.8 6533043.29999999),2,POINT(399179 6533036.79999999),78.0,2.0,"(D215, D117)","(172840091, 173890120, 173890140)",3.0,"(Double,)"
55,TRONROUT0000000100999103,"(LINESTRING (380571.6999999997 6568601.899999993, 380569.6999999997 6568601.699999994, 380567.3999999996 6568601.999999993))",210382,Route à 1 chaussée,NC,3,Autre,NC,NR,Non,NC,5.0,4.6,4.6,2.0,Inverse,NR,0.004329,estimation,None,None,None,None,/2,6720.0,...,None,None,None,46.0,40.0,111+,None,None,None,4652,25344,None,None,173002865,NaN,3,POINT(380571.7 6568601.89999999),3,POINT(380567.4 6568601.99999999),79.0,3.0,"(NC,)","(173004512, 173007016, 173002865)",8.0,"(Inverse, Direct)"
77,TRONROUT0000000032865847,"(LINESTRING (395698.1999999997 6563555.599999991, 395696.1999999998 6563555.099999993, 395689.9999999997 6563552.199999993, 395685.5999999996 6563548.099999992))",37814,Route à 1 chaussée,D939,3,Départementale,17,NR,Non,NC,7.0,23.7,23.7,2.0,Direct,NR,0.014920,linearisation,otv,17-D939-79+729,None,2015,/2,6649.0,...,None,None,None,46.0,40.0,111+,None,None,None,19997,93197,None,None,NR,NaN,3,POINT(395698.2 6563555.59999999),3,POINT(395685.6 6563548.09999999),117.0,3.0,"(D939, D5, D204E3)","(17003B048, NR)",8.0,"(Inverse, Direct)"
91,TRONROUT0000000032933286,"(LINESTRING (378959.9999999997 6534842.999999992, 378966.5999999997 6534847.899999994, 378971.2999999996 6534854.699999992, 378972.9999999996 6534862.499999991))",100158,Route à 1 chaussée,D26,2,Départementale,17,NR,Non,NC,6.0,7.1,7.1,2.0,Direct,NR,0.024469,linearisation,otv,17-D26-2+174,None,2015,/2,15785.0,...,None,None,None,57.0,52.0,None,None,None,None,86299,86300,None,None,NR,NaN,3,POINT(378960 6534842.99999999),3,POINT(378973 6534862.49999999),138.0,3.0,"(D26, D728, NC)","(17058#00G, NR, 170580720, 17058#013)",8.0,"(Direct,)"
101,TRONROUT0000000032956900,"(LINESTRING (379523.9999999997 6525978.799999991, 379528.8999999996 6525983.499999992, 379531.7999999996 6525989.599999991, 379532.3999999996 6525996.399999992))",122038,Route à 1 chaussée,D14,2,Départementale,17,NR,Non,NC,7.0,6.5,6.3,2.0,Direct,NR,0.020370,linearisation,otv,17-D14-16+600,None,2015,/2,5406.0,...,None,None,None,91.0,80.0,None,None,None,None,90506,90349,None,None,NR,NaN,3,POINT(379524 6525978.79999999),3,POINT(379532.4 6525996.39999999),147.0,2.0,"(NC, D14)","(174520171, NR, 170210195, 170210022)",8.0,"(Inverse, Direct)"
102,TRONROUT0000000032956945,"(LINESTRING (378298.5999999996 6525959.199999991, 378302.3999999997 6525963.599999991, 378307.0999999996 6525966.999999992))",122080,Route à 1 chaussée,D14,2,Départementale,17,NR,Non,NC,7.0,10.7,10.6,2.0,Inverse,NR,0.011615,linearisation,otv,17-D14-16+600,None,2015,/2,5406.0,...,None,None,gsw,57.0,52.0,None,None,None,None,35288,63926,None,None,NR,NaN,3,POINT(378298.6 6525959.19999999),3,POINT(378307.1 6525966.99999999),148.0,4.0,"(NC, D14E1, D25, D14)","(NR, 170210156, 170210062)",9.0,"(Inverse, Direct, Double)"
103,TRONROUT0000000032956946,"(LINESTRING (378353.8999999997 6525935.199999993, 378353.4999999997 6525932.499999992, 378351.5999999996 6525927.899999991, 378349.1999999997 6525923.099999993))",122081,Route à 1 c

In [36]:
carac_rd_pt.reset_index().dtypes

id_rdpt                int64
nb_rte_rdpt            int64
nom_rte_rdpt          object
codevoie_rdpt         object
nb_obj_sig_entrant     int64
valeur_sens           object
dtype: object

# SUITE

In [114]:
toto=list(at.recup_troncon_elementaire('TRONROUT0000000032862365',df,[]))

In [115]:
toto.append('TRONROUT0000000032862365')

In [ ]:
toto

In [117]:
at.recup_troncon_parallele_v2(df,toto)

C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees\agrege_troncon.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))


'TRONROUT0000000032876496'

In [ ]:
for i in toto :
    tata=at.affecter_troncon_ligne(i,df,df2_chaussees)
    print(len(tata))

In [12]:
dico=at.affecter_troncon(df,df2_chaussees)

0eme occurence : TRONROUT0000000084053027 à 18:23:53 nb ligne traite : 0, nb ligne differente=0
indexError ligne  TRONROUT0000000032954220
indexError ligne  TRONROUT0000000033016282
indexError ligne  TRONROUT0000000249471326
indexError ligne  TRONROUT0000000336640149
indexError ligne  TRONROUT0000000224966440
indexError ligne  TRONROUT0000000032833606
indexError ligne  TRONROUT0000000032895598
indexError ligne  TRONROUT0000000032991823
indexError ligne  TRONROUT0000000223846575
indexError ligne  TRONROUT0000000255628539
indexError ligne  TRONROUT0000000032895522
indexError ligne  TRONROUT0000000032843156
indexError ligne  TRONROUT0000000032966066
indexError ligne  TRONROUT0000000129676068
indexError ligne  TRONROUT0000000032855812
indexError ligne  TRONROUT0000000033029153
indexError ligne  TRONROUT0000000323460798
indexError ligne  TRONROUT0000000032937456
indexError ligne  TRONROUT0000000089679809
indexError ligne  TRONROUT0000000211249489
5000eme occurence : TRONROUT0000000032859682

In [13]:
with ct.ConnexionBdd('local_otv') as c :
    at.inserer_dico(c, dico)

POUR TRAITEMENT DES VOIES A 2 CHAUSSES : TROUVER PARRALLELE

In [87]:
#liste des troncons (parametre entrant)
liste_troncon=toto

In [88]:
#conversion en geodf
gdf_lignes=gp.GeoDataFrame(df.loc[liste_troncon], geometry='geom')
gdf_lignes.to_file(r'D:\temp\otv\17\2_chaussees\lignes_a_gouper.shp')

In [89]:
#union des geometries
gdf_lignes2=linemerge(gdf_lignes.unary_union)

In [ ]:
gdf_lignes2.wkt

In [90]:
#centroid de la ligne
xmin,ymin,xmax,ymax=gdf_lignes2.interpolate(0.5, normalized=True).buffer(50).bounds
point=gdf_lignes2.interpolate(0.5, normalized=True)

In [91]:
point.wkt

'POINT (385035.1182861876 6561685.156661048)'

In [92]:
#recherche des lignes proches du centroid
lignes_possibles=df.cx[xmin:xmax,ymin:ymax]
lignes_possibles.to_file(r'D:\temp\otv\17\2_chaussees\lignes_possibles.shp')

In [ ]:
lignes_possibles

In [94]:
#uniquement les lignes non présentes dans la liste de troncons avec le même nom de voie
ligne_filtres=lignes_possibles.loc[(~lignes_possibles.index.isin(liste_troncon)) & (lignes_possibles['numero']==gdf_lignes.iloc[0]['numero'])]

In [95]:
#obtenir les distances au centroid
ligne_filtres['distance']=ligne_filtres.geom.apply(lambda x : gdf_lignes2.centroid.distance(x))

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
#garder uniquement la valeur la plus proche du centroid
ligne_proche=ligne_filtres.loc[ligne_filtres['distance']==ligne_filtres['distance'].min()].index.tolist()[0]
ligne_proche

'TRONROUT0000000129676313'

POUR TRAITEMENT DES ROND POINTS

In [4]:
#polygoniser les lignes
#creer la gdf
df.crs={'init':'epsg:2154'}

In [15]:
#creer la lsite des rd points selon le critere atmo aura ; creer une gdf avec la liste
liste_rd_points=[t.buffer(0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14] # car un cercle à un rappor de ce type entre 12 et 13
dico_rd_pt=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point=gp.GeoDataFrame(dico_rd_pt, geometry=liste_rd_points)
gdf_rd_point.crs={'init':'epsg:2154'}
gdf_rd_point.columns=['id_rdpt', 'facteur','geometry']
#on créer aussi la meme donnees avec un buffer interiuer, pour ne garder que les lignes dans le buffer exteriuer et hors buffer interieur (cas de rond point enjmabeant uine 2*2 et prenant la 2*2 voie qui est dans le polygone
liste_rd_points_int=[t.buffer(-0.1) for t in polygonize(df.geometry) if 12<=((t.length**2)/t.area)<=14]
dico_rd_pt_int=[[i, ((t.length**2)/t.area)] for i,t in enumerate(polygonize(df.geometry)) if 12<=((t.length**2)/t.area)<=14]
gdf_rd_point_int=gp.GeoDataFrame(dico_rd_pt_int, geometry=liste_rd_points_int)
gdf_rd_point_int.crs={'init':'epsg:2154'}
gdf_rd_point_int.columns=['id_rdpt', 'facteur','geometry']

In [63]:
#jointure spataile pour une gdf avec uniquement les lignes des rd_points avec le numéro
l_dans_p=gp.sjoin(df,gdf_rd_point,op='within') 

In [64]:
#suprimer les doublons
l_dans_p_ss_doublon=l_dans_p.reset_index().rename(columns={'index':'id_ign'}).drop_duplicates('id_ign').set_index('id_ign')

In [66]:
l_dans_p_int=gp.sjoin(df,gdf_rd_point_int,op='within')

In [67]:
l_dans_p_final=l_dans_p_ss_doublon.loc[~l_dans_p_ss_doublon.index.isin(l_dans_p_int.index.tolist())]

In [68]:
#lignes qui touchent rd points
#1.ligne qui intersectent avec id_rdpt
l_intersct_rdpt=gp.sjoin(df,l_dans_p_final.drop('index_right', axis=1), how='inner',op='intersects')

In [69]:
#2.filtre de celle contenue dans le rd points 
l_intersct_rdpt=l_intersct_rdpt.loc[~l_intersct_rdpt.index.isin(l_dans_p_final.index.tolist())][['id_rdpt','numero_left','codevoie_d_left']]

In [70]:
#trouver le nb de voies qui intersectent chaque rd point et leur noms. renomer les colonnes
carac_rd_pt=(pd.concat([l_intersct_rdpt.groupby('id_rdpt').numero_left.nunique(),
    l_intersct_rdpt.groupby('id_rdpt')['numero_left'].apply(lambda x: ','.join(set(x))),
    l_intersct_rdpt.groupby('id_rdpt')['codevoie_d_left'].apply(lambda x: ','.join(set(x)))], axis=1))
carac_rd_pt.columns=['nb_rte_rdpt', 'nom_rte_rdpt','codevoie_rdpt']

In [ ]:
l_dans_p_final

In [297]:
#ajouter l'id_rdpt aux données
gdf_l_rd_point=pd.concat([df.drop('id_rdpt',axis=1),l_dans_p_final.loc[:,'id_rdpt']],axis=1, sort=False)

In [298]:
#mettre à jour la nature
gdf_l_rd_point['nature']=gdf_l_rd_point.apply(lambda x : 'Rd_pt' if x.id_rdpt>=0 else x['nature'], axis=1)

In [299]:
#ajouter les infos du rd point (nb voies différentes et nom)
gdf_avec_rd_pt=gdf_l_rd_point.merge(carac_rd_pt, how='left',left_on='id_rdpt', right_index=True)

In [320]:
def assigne_rdpt(id_rdpt,numero, liste_num, codevoie, liste_code) : 
    if id_rdpt >=0 :
        if (numero in liste_num) or (codevoie in liste_code) :
            return True
        else : 
            return False

In [321]:
gdf_avec_rd_pt['assigne_rdpt']=gdf_avec_rd_pt.apply(lambda x : assigne_rdpt(x['id_rdpt'],x['numero'], x['nom_rte_rdpt_y'], x['codevoie_d'],x['codevoie_rdpt']),axis=1)

In [ ]:
#gdf_avec_rd_pt.loc[gdf_avec_rd_pt.loc[:,'id_rdpt']==l_dans_p['id_rdpt']]
gdf_avec_rd_pt['id_rdpt']==l_dans_p['id_rdpt'].astype('float64')
#l_dans_p.loc[l_dans_p.loc[:,'id_rdpt']].sort_index().sort_index(axis=1)])

In [ ]:
#pour export et verif uniquement

#gdf_l_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test2.shp')
#gdf_rd_point.to_file(r'D:\temp\otv\17\test_rd_point\test1.shp')
#l_intersct_rdpt.to_file(r'E:\Boulot\otv\test_linearisation_auto\test3.shp')
df.to_file(r'E:\Boulot\otv\test_linearisation_auto\test7.shp')
#df.to_file(r'D:\temp\otv\17\test_rd_point\test5.shp')

In [91]:
','.join(set(['a','a','b','c']))

'b,c,a'

In [228]:
id_ign_ligne='TRONROUT0000000342862858'
df_ligne = df.loc[id_ign_ligne]
ligne_traite_troncon=[]
ligne_traite_troncon.append(id_ign_ligne)
liste_ligne_suivantes=[] 
liste_ligne_touchees=[]

In [ ]:
for key, value in {'nb_intrsct_src':['source', 'src_geom'],'nb_intrsct_tgt':['target', 'tgt_geom']}.items() :
    df_touches_source = df.loc[(~df.index.isin(ligne_traite_troncon)) & ((df['source'] == df_ligne[value[0]]) | (df['target'] == df_ligne[value[0]]))]
    df_rd_pt=df.loc[df['id_rdpt']==df_touches_source.iloc[0]['id_rdpt']]
    l_df_rd_pt=df.loc[df.index.isin(gp.sjoin(df,df_rd_pt, how='inner',op='intersects').index.unique().tolist())]
    l_df_rd_pt2=l_df_rd_pt.loc[~l_df_rd_pt.index.isin(df_rd_pt.index.tolist())].index.unique().tolist()
    l_df_rd_pt3=df.loc[df.index.isin(l_df_rd_pt2),'codevoie_d'].unique().tolist()
    print(l_df_rd_pt3)
    #liste_ligne_touchees+=df_touches_source.index.tolist()

In [233]:
df_touches_source['id_rdpt'].values.tolist()

[nan]